# Обучение. Инициализация, оптимизаторов, batch-norm

## Улучшение сходимости

Ускорение сходимости


* Инициализация (Xavier, He)

* Нормализация (Batch Normalization, Layer normalization)

Борьба с переобучением


* Регуляризация (Dropout, DropConnect)

## Инициализация весов

### Xavier (Glorot)

Рассмотрим нечетную функцию с единичной производной в нуле в
качестве активации (напр. tanh)

Хотим начать из линейного региона, чтобы избежать
затухающих градиентов

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-04.png" width="600">

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-05.png" width="500">

### He 

Рассмотрим ReLU в качестве активации:
* Функция не симметрична
* Не дифференцируема в нуле

ReLU или Rectified Linear Unit стала довольно популярной в последние годы. Она вычисляет функцию f(x) = max(0,x), то есть просто выдаёт значения «ноль» и «не ноль». Это решает проблему обнуления градиента для положительных чисел. Кроме того, ReLU очень просто вычисляется: примерно в шесть раз быстрее сигмоиды и тангенса. Однако, в ней снова отсутствует нулевое центрирование.

Другой очевидный недостаток — градиент по-прежнему «умирает» при отрицательных входных данных. Это может привести к тому, что половина нейронов будет неактивна и не сможет обновляться. 

Проблему можно попробовать решить, задав более низкую скорость обучения и подобрав другие весовые коэффициенты. Или использовать модификации ReLU.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-06.png" width="600">

### Xavier против He для ReLU

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-07.png" width="500">

Когда было обучение более глубоких сети, в которых использовался ReLU, He et. и др. было обнаружено, что 30-слойный CNN, использующий инициализацию Xavier, полностью застопорился и не научился вообще Однако, когда одна и та же сеть была инициализирована в соответствии с трехэтапной процедурой, описанной выше, она имела значительно большую конвергенцию.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-08.png" width="500">

Сходимость 30-слойCNN благодаря Kaiming init. Кредит

ссылка https://arxiv.org/pdf/1502.01852.pdf

Мораль этой истории для нас заключается в том, что любая сеть, которую мы обучаем с нуля, особенно для приложений компьютерного зрения, почти наверняка будет содержать функции активации ReLU и иметь несколько уровней. В таких случаях Kaiming должен быть нашей стратегией инициации.

Вот тут есть вопрос-ответ с кодом, может понадобится

**Ручной инициализатор Xavier: какие значения для lrelu и relu**

В качестве продолжения ответа (не выбранного) в How to do Xavier initialization on TensorFlow: у кого-нибудь есть идея, какие значения использовать в relu и особенно leaky relu?

https://coderoad.ru/39231032/%D0%A0%D1%83%D1%87%D0%BD%D0%BE%D0%B9-%D0%B8%D0%BD%D0%B8%D1%86%D0%B8%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80-Xavier-%D0%BA%D0%B0%D0%BA%D0%B8%D0%B5-%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F-%D0%B4%D0%BB%D1%8F-lrelu-%D0%B8-relu

### Ортогональная инициализация

*Инициализатор, который генерирует случайную ортогональную матрицу*

Выберем ортогональную матрицу весов W: WWT = I. Тогда:
* ∥*Wix*∥ = ∥*x*∥ — норма сохраняется
* ⟨*Wi, Wj*⟩ = *δij* — все нейроны делают «разные» преобразования

Что делать для сверточных слоев?

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-10.png" width="600">

## Регуляризация

### Dropout

Регуляризация используется для упрощения модели, чтобы избежать переобучения и заставить её корректно работать на новых данных. Но чаще всего в машинном обучении применяется эффективный и простой алгоритм Dropout.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-12.png" width="500">

Каждый раз, выполняя прямой проход по слою, мы устанавливаем некоторые значения активаций нулевыми (нейроны выбираются случайно). В результате мы получим несколько «подсетей» и будем поочерёдно оценивать их эффективность, что делает dropout похожим на ансамбль методов.

* С вероятностью *p* занулим выход нейрона (например, *p* = 0.5) 
* В test-time домножаем веса на вероятность сохранения
* Не стоит выкидывать нейроны последнего слоя

**Самая простая реализация для трёхслойной нейросети выглядит следующим образом:**

In [ ]:
""" Vanilla Dropout: не рекомендуется использовать на практике """

p = 0.5 # вероятность оставить нейрон активным. Чем выше, тем меньше dropout

def train_step(X):

  """ X - данные """

  # прямой проход по трёхслойной нейросети

  H1 = np.maximum(0, np.dot(W1, X) + b1)

  U1 = np.random.rand(*H1.shape) < p # первый dropout

  H1 *= U1 # drop!

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  U2 = np.random.rand(*H2.shape) < p # второй dropout

  H2 *= U2 # drop!

  out = np.dot(W3, H2) + b3

  # обратный проход: вычисление градиентов... (не показано)

  # обновление параметров... (не показано)

def predict(X):

  # ансамблевый прямой проход

  H1 = np.maximum(0, np.dot(W1, X) + b1) * p # масштабируем активации!

  H2 = np.maximum(0, np.dot(W2, H1) + b2) * p # масштабируем активации!

  out = np.dot(W3, H2) + b3

Нежелательное свойство представленной выше схемы состоит в том, что мы должны масштабировать активации во время прогнозирования. Поскольку производительность тестирования критически важна, лучше всего использовать инвертированный dropout, который выполняет масштабирование во время обучения. Кроме того, если мы захотим убрать dropout из кода, функция прогнозирования останется без изменений.

In [ ]:
""" 

Inverted Dropout: Рекомендуемая реализация

"""

p = 0.5 # вероятность оставить нейрон активным. Чем выше, тем меньше dropout

def train_step(X):

  # прямой проход по трёхслойной нейросети

  H1 = np.maximum(0, np.dot(W1, X) + b1)

  U1 = (np.random.rand(*H1.shape) < p) / p # первый dropout. Заметьте /p!

  H1 *= U1 # drop!

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  U2 = (np.random.rand(*H2.shape) < p) / p # второй dropout. Заметьте /p!

  H2 *= U2 # drop!

  out = np.dot(W3, H2) + b3

  # обратный проход: вычисление градиентов... (не показано)

  # обновление параметров... (не показано)

def predict(X):

  # ансамблевый прямой проход

  H1 = np.maximum(0, np.dot(W1, X) + b1) # масштабирование не нужно

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  out = np.dot(W3, H2) + b3

Dropout очень полезен тем, что с его помощью можно менять силу регуляризации, просто варьируя значение параметра p.

### Dropout, мотивация

Dropout изменил концепцию обучения всех весов вместе, чтобы узнать долю весов в сети на каждой итерации обучения.

Одной из основных проблем в изучении больших сетей является совместная адаптация. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

В таком сценарии, поскольку сеть обучается итеративно, эти мощные соединения изучаются больше, в то время как более слабые игнорируются. За многие итерации обучается только часть соединений узлов. А остальные перестают участвовать.

Это явление называется со-адаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. Причина в том, что они также упорядочены на основе прогнозирующих возможностей соединений. Благодаря этому они становятся близки к детерминистическим в выборе и отклонении весов. И, таким образом, снова сильный становится сильнее, а слабый становится слабее.

Основные последствия этого были:расширение размера нейронной сети не поможет, Следовательно, размер нейронных сетей и, следовательно, точность стали ограниченными.

Затем пришел Dropout. Новый подход к регуляризации. Это решило совместную адаптацию. Теперь мы могли бы строить более глубокие и широкие сети. И используйте силу предсказания всего этого.

На этом фоне давайте погрузимся в математику отсева. Вы можете перейти непосредственно кОтсев, эквивалентный регуляризованной сетираздел для выводов,

* Борьба с соадаптацией – нейроны больше не могут рассчитывать на наличие соседей
* Биология: не все гены родителей будут присутсвовать у
потомков
* Усреднение большого (2^*n*) числа моделей

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-13.png" width="400">

**Figure:** Выученные признаки на MNIST (автокодировщик с одним скрытым слоем и ReLU в качестве активации). 

Слева: без Dropout, справа – с Dropout

**Зачем мне калечить (cripple) нейронную сеть?**


Как бы странно это ни звучало, отмена способности некоторых нейронов учиться во время тренировки на самом деле направлена на то, чтобы получить более обученные нейроны и сократить переобучение.

Таким образом, мы получаем приблизительный результат усреднения более простых обученных моделей, которые в противном случае заняли бы гораздо больше времени и вычислительной мощности, чтобы обучаться по очереди. Но это не единственная причина. На самом деле, тренировка сетей таким образом не только помогает им адаптироваться, уравновешивая их слабые и сильные стороны, но и гарантирует, что инкапсулируемые ими функции хорошо работают со случайно выбранными подмножествами усвоенных функций других.

Это приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. На следующем рисунке (извлеченном из статьи Dropout: A Simple Way to Prevent Neural Networks from Overfitting) мы находим сравнение признаков, изученных в наборе данных MNIST с одним автоэнкодером скрытого слоя, имеющим 256 выпрямленных линейных единиц без отсева (слева), и признаков, изученных той же структурой с использованием отсева в ее скрытом слое с p=0,5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

Источник https://quantdare.com/dropout-in-feed-forward-neural-networks/

перевод яндекс переводчика, при необходимости перепишу человеческим текстом

### Dropconnect

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-14.png" width="700">

**Зануляем не выходы нейронов, а каждый вес по отдельности**

Drop Connect обобщает Drop out, случайным образом отбрасывая веса, а не активации с вероятностью *1 — p*. DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель , но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. Обратите внимание, что это не эквивалентно *W* lkz установкb фиксированной разреженной матрицы во время обучения.

Для слоя Drop Connect выходные данные задаются следующим образом:

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-14-1.png" width="300">

Где *r* выход слоя, *u* это вход в слой, *W* это весовые параметры и *M* это двоичная матрица, кодирующая информацию о соединении где *M(ij) ~ Bernoulli(p)*. Каждый элемент матрицы *M* создается независимо для каждого примера во время обучения, по существу создавая различные связи для каждого увиденного примера. Кроме того, предубеждения также маскируются во время обучения.

Источник https://paperswithcode.com/method/dropconnect

перевод яндекс переводчика, при необходимости перепишу человеческим текстом

## Нормализация

Рассмотрим классическую нейронную сеть с несколькими слоями. Каждый слой имеет множество входов и множество выходов. Сеть обучается методом обратного распространения ошибки, по батчам, то есть ошибка считается по какому-то подмножестве обучающей выборки.


Стандартный способ нормировки — для каждого k рассмотрим распределение элементов батча. Вычтем среднее и поделим на дисперсию выборки, получив распределение с центром в 0 и дисперсией 1. Такое распределение позволит сети быстрее обучатся, т.к. все числа получатся одного порядка. Но ещё лучше ввести две переменные для каждого признака, обобщив нормализацию следующим образом:

### Алгоритм

Входы: Значения x в мини-батче B = {xi}
m
i=1;

Параметры: γ, β
Выход: {yi = BNγ,β(xi)}

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-19.png" width="600">

Получим среднее, дисперсию. Эти параметры будут входить в алгоритм обратного распространения ошибки.
Тем самым получаем batch normalization слой с 2*k параметрами, который и будем добавлять в архитектуру предложенной сети для распознавания лиц.


На вход в моей задаче подаётся черно-белое изображение лица человека размером 50x50 пикселей. На выходе имеем 14000 вероятностей классов. Класс с максимальной вероятностью считается результатом предсказания.

### Мотивация

* Обычно наблюдается более быстрая сходимость при
декорелированных входах
* Whitening: xˆ = Cov[x]−1/2(x − E[x])

* Нормализация: xˆ
(k) =
x
(k)−E[x
(k)
√
]
V ar[x(k)]
для каждой размерности

### Батч-нормализация

* Covariate shift: изменение распределения входов во время
обучения
* Цель — уменьшить covariate shift скрытых слоев
* Нормализуем входы в каждый слой xˆ
(k) =
x
(k)−E[x
(k)
√
]
D[x(k)]
* Статистики Ex и Dx оценим для каждого мини-батча



**? Почему этот метод плох для сетей с сигмоидами?**

* Сигмоиды становятся почти линейными ⇒ линейная модель :(
* Доп. параметры: y
^(k) = γ
^(k)xˆ
^(k) + β
^(k)

### Градиент

Можно вычислить градиент при помощи chain rule

Важно помнить, что *μB* и *σ
2
B*
не являются константами

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-20.png" width="700">

### Предсказание

Во время предсказания батч-нормализация является линейным
слоем:

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-21.png" width="500">

E[*x*] и D[*x*] вычисляются по всему обучающему множеству.
На практике статистики вычисляются во время обучения
экспоненциальным средним: *Ei+1* = (1 − *α*)*Ei* + *αEB*

### Batchnorm как регуляризация

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-22.png" width="400">

При увеличении весов в *a* раз, градиент выхода слоя по входу не меняется, а градиент по весам уменьшается в *a* раз.

### Tips

Стоит помнить, что с батч-нормализацией:
* Надо убрать смещения
* Другое расписание learning rate: большее значение в начале обучения и быстрое уменьшение в процессе обучения
* Уменьшить силу Dropout и *L2* регуляризации
* Перемешивать обучающую выборку

Lля изображений: нормализация каждого канала (одинаковые среднее и дисперсия вдоль пространственных размерностей)

### Обучение

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-24.png" width="600">

### Layer Normalization

До этого момента мы рассматривали способы навигации по поверхности потерь нейронной сети с использованием импульса и адаптивных скоростей обучения. Мы также рассмотрели несколько методов инициализации параметров, чтобы минимизироватьаприориуклоны внутри сети. В этом разделе мы рассмотрим, как мы можем манипулировать самими данными, чтобы помочь нашей оптимизации модели.

При пакетной нормализации происходит усреднение параметров по всему пакету. Например, в случае задачи переноса стилей картин, это вносит много шума. При усреднении теряются индивидуальные характеристики объектов. Поэтому используется более тонкая нормализация — индивидуальная нормализация (англ. instance normalization). 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-25.png" width="700">

*Типы нормализации. Ось N — по объектам в пакете, ось C — по картам признаков (channels), оставшаяся ось — по пространственным измерениям объектов, например, ширине и высоте картинки.*

Разница заключается в том, что нормализация происходит по каждому отдельному объекту, а не по всему пакету. Для примера, усреднение происходит по пикселям картины, но не по всем картинам в пакете, как видно на рисунке.